# Gaussian Elimination

## by Dion Ho

### Last updated: February 14, 2019


# Import Python Libraries

In [1]:
import numpy as np
import math
import scipy.sparse as sp
from math import pi
from math import factorial
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def LU_orig(A):
    m = A.shape[1]
    U = A.copy()
    L = np.eye(m)
    for k in range(1,m):
        for j in range(k+1,m+1):
            L[j-1,k-1] = U[j-1,k-1]/U[k-1,k-1]
            U[j-1,(k-1):] = U[j-1,(k-1):] - L[j-1,k-1]*U[k-1,(k-1):]
    return L, U

In [3]:
def LU_modified(B):
    A = B*(1.0)
    m = A.shape[1]
    U = A.copy()
    L = np.eye(m)
    for k in range(1,m):
        L[k:, k-1] = U[k:,k-1]/U[k-1,k-1]
        U[k:, (k-1):] = U[k:, (k-1):] - np.outer(L[k:,k-1], U[k-1, (k-1):])
    return L, U

In [4]:
def LU_modified_pivot(B):
    A = B*(1.0)
    m = A.shape[1]
    U = A.copy()
    L = np.eye(m)
    P = np.eye(m)
    """again = False"""
    for k in range(1,m):
        maxVal = abs(U[k-1,k-1])
        maxIndex = k-1
        for ii in range(k,m):
            if abs(U[ii,k-1]) > maxVal:
                maxVal = abs(U[ii,k-1])
                maxIndex = ii
        if maxIndex != k-1:
            """again = True"""
            P[k-1],P[maxIndex] = np.copy(P[maxIndex]),np.copy(P[k-1])
            U[(k-1),(k-1):],U[maxIndex,(k-1):] = np.copy(U[maxIndex,(k-1):]),np.copy(U[(k-1),(k-1):])
            L[(k-1),:(k-1)],L[maxIndex,:(k-1)] = np.copy(L[maxIndex,:(k-1)]),np.copy(L[(k-1),:(k-1)]) #OMG it works now!
        L[k:, k-1] = U[k:,k-1]/U[k-1,k-1]
        U[k:, (k-1):] = U[k:, (k-1):] - np.outer(L[k:,k-1], U[k-1, (k-1):])
    
    """if again == True: #Extremely crude, but I couldn't modify L correctly, so I just repeat the Gaussian Elimination after we know exactly how to permute A.
        U = (P@A).copy()
        L = np.eye(m)
        for k in range(1,m):
            L[k:, k-1] = U[k:,k-1]/U[k-1,k-1]
            U[k:, (k-1):] = U[k:, (k-1):] - np.outer(L[k:,k-1], U[k-1, (k-1):])"""
    return L, U, P

In [5]:
D = np.array([[2,1,1,0],[4,3,3,1], [8,7,9,5], [6,7,9,8]])
print("Original matrix")
print(D)
print()
L1,U1 = LU_modified(D)
print("Matrix L")
print(L1)
print("Matrix U")
print(U1)
print("LU_modified: Matrix formed by multiplication of L with U.")
print(L1@U1) #LU_modified works as intended.
L1p,U1p,P1 = LU_modified_pivot(D)
print("LU_modified_pivot: Matrix formed by multiplication of L with U.")
print(P1)
print(np.matrix.transpose(P1)@L1p@U1p) #Since this actually solves PA = LU, we need to multiply P inverse with LU; the inverse of a permutation matrix is just its transpose.

Original matrix
[[2 1 1 0]
 [4 3 3 1]
 [8 7 9 5]
 [6 7 9 8]]

Matrix L
[[1. 0. 0. 0.]
 [2. 1. 0. 0.]
 [4. 3. 1. 0.]
 [3. 4. 1. 1.]]
Matrix U
[[2. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 0. 2. 2.]
 [0. 0. 0. 2.]]
LU_modified: Matrix formed by multiplication of L with U.
[[2. 1. 1. 0.]
 [4. 3. 3. 1.]
 [8. 7. 9. 5.]
 [6. 7. 9. 8.]]
LU_modified_pivot: Matrix formed by multiplication of L with U.
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
[[2.00000000e+00 1.00000000e+00 1.00000000e+00 1.11022302e-16]
 [4.00000000e+00 3.00000000e+00 3.00000000e+00 1.00000000e+00]
 [8.00000000e+00 7.00000000e+00 9.00000000e+00 5.00000000e+00]
 [6.00000000e+00 7.00000000e+00 9.00000000e+00 8.00000000e+00]]


In [7]:
D = np.random.randint(0,9,(5,5)) #With a random array, just to make sure it works.
print("Original matrix")
print(D)
print()
L2,U2 = LU_modified(D)
print("Matrix L")
print(L2)
print("Matrix U")
print(U2)
print("LU_modified: Matrix formed by multiplication of L with U.")
print(L2@U2) #LU_modified works as intended.
L2p,U2p,P2 = LU_modified_pivot(D)
print("LU_modified_pivot: Matrix formed by multiplication of L with U.")
print(np.matrix.transpose(P2)@L2p@U2p) #Since this actually solves PA = LU, we need to multiply P^-1 with LU; the inverse of a permutation matrix is just its transpose.

Original matrix
[[8 0 5 8 3]
 [3 2 2 7 0]
 [3 6 5 3 6]
 [2 3 6 1 6]
 [0 2 4 8 0]]

Matrix L
[[1.         0.         0.         0.         0.        ]
 [0.375      1.         0.         0.         0.        ]
 [0.375      3.         1.         0.         0.        ]
 [0.25       1.5        1.65909091 1.         0.        ]
 [0.         1.         1.40909091 1.61971831 1.        ]]
Matrix U
[[  8.           0.           5.           8.           3.        ]
 [  0.           2.           0.125        4.          -1.125     ]
 [  0.           0.           2.75       -12.           8.25      ]
 [  0.           0.           0.          12.90909091  -6.75      ]
 [  0.           0.           0.           0.           0.43309859]]
LU_modified: Matrix formed by multiplication of L with U.
[[ 8.00000000e+00  0.00000000e+00  5.00000000e+00  8.00000000e+00
   3.00000000e+00]
 [ 3.00000000e+00  2.00000000e+00  2.00000000e+00  7.00000000e+00
   0.00000000e+00]
 [ 3.00000000e+00  6.00000000e+00  5.00